# Matrix Multiplication

We will use multithreading/multiprocessing to compute matrix products. We will use a thread/process to compute the next row of the resulting matrix. 


First, we will use multithreading. We add a sleep time in the task function to simulate an I/O process and show the interest of multithreading. Then we will use multiprocessing and show the interest for a CPU bound task.

In [3]:
import time
import random
import threading
import numpy as np
from time import perf_counter

In [4]:
def timeit(func):
    """ decorator that times a function """
    def inner(*args, **kwargs):
        start = perf_counter()
        res = func(*args, **kwargs)
        end = perf_counter()
        print(f"time taken {end-start:.2f}s")
        return res
    return inner

In [5]:
def sleep_for(n):
    """ decorator factory that takes a sleeping time as input """
    def decorator(func):
        """ decorator """
        def inner(*args, **kwargs):
            time.sleep(n)
            return func(*args, **kwargs)
        return inner
    return decorator

In [6]:
def gen_random_mat(n):
    """ Generate random matrice of size n"""
    mat = [ [random.randint(1,5) for _ in range(n)] for _ in range(n)]
    return mat

def render_mat(mat):
    """ render matrice """
    for row in mat:
        print(row)

def verify_with_numpy(matA, matB):
    """ Verify result with numpy calculations """
    return np.matmul(np.array(matA), np.array(matB))

def row_work(args):
    """ return product of row time a matrice """
    res = []
    row, mat_B = args
    for j in range(len(mat_B)):
        a = 0
        for i in range(len(row)):
            a += row[i]*mat_B[i][j]
        res.append(a)
    return res   

In [7]:
mat_A = gen_random_mat(4)
mat_B = gen_random_mat(4)

print("mat_A")
render_mat(mat_A)

print()

print("mat_B")
render_mat(mat_B)

mat_A
[4, 2, 5, 5]
[3, 4, 4, 3]
[3, 4, 1, 5]
[2, 1, 3, 2]

mat_B
[1, 3, 5, 3]
[5, 1, 5, 2]
[3, 2, 3, 3]
[1, 1, 3, 1]


In [8]:
mat = []
for row in mat_A:
    res = row_work(args=(row, mat_B))
    mat.append(res)
render_mat(mat)

[34, 29, 60, 36]
[38, 24, 56, 32]
[31, 20, 53, 25]
[18, 15, 30, 19]


In [9]:
verify_with_numpy(mat_A, mat_B)

array([[34, 29, 60, 36],
       [38, 24, 56, 32],
       [31, 20, 53, 25],
       [18, 15, 30, 19]])

## I. I/O bound task: Multithreaded vs Non threaded

### a. Non Threaded

In [12]:
IO_row_work = sleep_for(1)(row_work)

In [13]:
# Non Threaded
@timeit
def main():
    mat = []
    for row in mat_A:
        res = IO_row_work((row, mat_B))
        mat.append(res)
    render_mat(mat)

In [14]:
main()

[34, 29, 60, 36]
[38, 24, 56, 32]
[31, 20, 53, 25]
[18, 15, 30, 19]
time taken 4.00s


### b. Multi Threaded with the concurrent module

In [16]:
import concurrent

In [17]:
@timeit
def main():
    # the pool made the decision on how to affect worker
    with concurrent.futures.ThreadPoolExecutor() as executor:
    
        args = ((row,mat_B) for row in mat_A) # generator so it is lazy
        results = executor.map(IO_row_work, args)
    
        # iterator than we can loop over that will yield result in execution order
        mat = [res for res in results]
        
        render_mat(mat)
main()

[34, 29, 60, 36]
[38, 24, 56, 32]
[31, 20, 53, 25]
[18, 15, 30, 19]
time taken 1.00s


The multithreaded is 4 times faster than the non threaded solution because they are 4 row operation perform in separate threads.

## II. CPU bound task : Sequential vs Multithreaded vs Multiprocessed

In [20]:
n = 400
mat_A = gen_random_mat(n)
mat_B = gen_random_mat(n)

#### a. sequential

In [22]:
@timeit
def main():

    for t in range(10):
        mat = []
        for row in mat_A:
            res = row_work((row, mat_B))
            mat.append(res)
            
        print(f"product {t+1} done")

main()

product 1 done
product 2 done
product 3 done
product 4 done
product 5 done
product 6 done
product 7 done
product 8 done
product 9 done
product 10 done
time taken 80.33s


#### b. multithreaded

In [24]:
@timeit
def main():

    for t in range(10):
        
        # the pool made the decision on how to affect worker
        with concurrent.futures.ThreadPoolExecutor() as executor:
        
            args = ((row,mat_B) for row in mat_A) # generator so it is lazy
            results = executor.map(row_work, args)
        
            # iterator than we can loop over that will yield result in execution order
            mat = [res for res in results]

        print(f"product {t+1} done")

main()

product 1 done
product 2 done
product 3 done
product 4 done
product 5 done
product 6 done
product 7 done
product 8 done
product 9 done
product 10 done
time taken 75.57s


#### c. multiprocessed
We perform one full matrix opertaion in a process. Multiprocess can be run directly in Jupyter notebook. Function must be compiled first that's why we import code from the matrix_multiprocess module

In [26]:
from matrix_multiprocess import execute_in_multiprocess

In [27]:
execute_in_multiprocess(mat_A, mat_B)

time taken 51.10s


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]